In [80]:
!pip install gensim

^C


In [115]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embeddindg
#import gensim
#from gensim.models import Word2Vec
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\macsd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\macsd\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\macsd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [1]:
import pickle
with open('./tagged_db.pickle', 'rb') as f:
    dataset = pickle.load(f)
    

In [90]:
import pandas as pd
df = pd.DataFrame(dataset)
#df.set_index(name, drop=True, append=False, inplace=False, verify_integrity=False)
df = df.transpose()
df['new_col'] = range(1, len(df) + 1)
df = df.set_index("new_col", drop=True, append=False, inplace=False, verify_integrity=False)
df = df.drop(df[df.topic == "filosofia"].index)
df = df.drop(df[df.topic == "geologia"].index)
df = df.drop(df[df.topic == "informatica"].index)
df = df.drop(df[df.topic == "biologia"].index)
df = df.drop(df[df.topic == "nessuno"].index)
df.wordlist[1][2]

'poco'

In [91]:
for i, row in df.iterrows():
       row.wordlist =  ' '.join(row.wordlist)

In [187]:
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('italiano')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [188]:
def finalpreprocess(string):
    return lemmatizer(preprocess(string))
df['clean_text'] = df['wordlist'].apply(lambda x: finalpreprocess(x))
df.head()

,filename,wordlist,topic,clean_text,y
new_col,,,,,
1,000001.jpg,esercizio sale poco solubile determinare quali...,chimica,esercizio sale poco solubile determinare quali...,0
2,000003.jpg,denotare effettivamente qualche non chiaro tal...,matematica,denotare effettivamente qualche non chiaro tal...,2
3,000005.jpg,quanti domini elettronici dipende geometria qu...,chimica,quanti domini elettronici dipende geometria qu...,0
6,000008.jpg,una sostanza costituita idrogeno azoto dato al...,chimica,una sostanza costituita idrogeno azoto dato al...,0
8,000026.jpg,l'analisi una sostanza costituita azoto cloro ...,chimica,l analisi una sostanza costituita azoto cloro ...,0


In [207]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
a = le.fit_transform(df.topic)
df["y"] = a

In [208]:
X_train, X_test, y_train, y_test = train_test_split(df["clean_text"],df["y"],test_size=0.2,shuffle=True)

In [191]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

In [106]:
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  
#Predict y value for test dataset
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 

              precision    recall  f1-score   support

           0       0.84      0.74      0.79        66
           1       0.89      0.80      0.85       189
           2       0.84      0.93      0.88       260

    accuracy                           0.86       515
   macro avg       0.86      0.82      0.84       515
weighted avg       0.86      0.86      0.86       515

Confusion Matrix: [[ 49   4  13]
 [  4 152  33]
 [  5  14 241]]


In [112]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train_vectors_tfidf, y_train)
y_predict = clf.predict(X_test_vectors_tfidf)
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.95      0.61      0.74        66
           1       0.94      0.76      0.84       189
           2       0.79      0.97      0.87       260

    accuracy                           0.84       515
   macro avg       0.89      0.78      0.82       515
weighted avg       0.86      0.84      0.84       515

Confusion Matrix: [[ 40   2  24]
 [  1 144  44]
 [  1   8 251]]


In [114]:
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)  
#Predict y value for test dataset
y_predict = nb_tfidf.predict(X_test_vectors_tfidf)
y_prob = nb_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 


              precision    recall  f1-score   support

           0       0.96      0.35      0.51        66
           1       0.84      0.83      0.83       189
           2       0.80      0.93      0.86       260

    accuracy                           0.82       515
   macro avg       0.87      0.70      0.74       515
weighted avg       0.83      0.82      0.81       515

Confusion Matrix: [[ 23  14  29]
 [  1 157  31]
 [  0  17 243]]


In [193]:
X_train_vectors_tfidf.toarray().shape
#y_train.shape


(2056, 7585)

In [202]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(7585, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64,64)
        self.fc5 = nn.Linear(64,3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x)

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=7585, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=3, bias=True)
)


In [209]:
X_train = torch.from_numpy(X_train_vectors_tfidf.toarray())
X_test = torch.Tensor(X_test_vectors_tfidf.toarray())
y_train = torch.from_numpy(y_train.values.astype(np.float))
y_test = torch.Tensor((y_test.values.astype(np.float)))
training_dataset = torch.utils.data.TensorDataset(X_train, y_train)
testing_dataset = torch.utils.data.TensorDataset(X_test, y_test)
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=256, shuffle=False)
test_loader = torch.utils.data.DataLoader(testing_dataset, batch_size = 256, shuffle = False)

<ipython-input-209-903ce32d9a2b>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train = torch.from_numpy(y_train.values.astype(np.float))
<ipython-input-209-903ce32d9a2b>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_test = torch.Tensor((y_test.values.astype(np.float)))


In [210]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr= 0.001)

In [211]:
lo = []
for epoch in range(100):
    for data in train_loader:
        x, y = data
        net.zero_grad()
        x = x.view(-1,7585).float()
        output = net(x)
        loss = loss_function(output, y.type(torch.LongTensor)) # calc and grab the loss value
        loss.backward()
        optimizer.step()
    print(loss)
    lo.append(loss.detach().numpy())

<ipython-input-202-8e3d07ae977f>:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


tensor(2.9344, grad_fn=<NllLossBackward>)
tensor(1.6702, grad_fn=<NllLossBackward>)
tensor(1.1681, grad_fn=<NllLossBackward>)
tensor(0.9354, grad_fn=<NllLossBackward>)
tensor(0.6539, grad_fn=<NllLossBackward>)
tensor(0.5062, grad_fn=<NllLossBackward>)
tensor(0.4660, grad_fn=<NllLossBackward>)
tensor(0.4542, grad_fn=<NllLossBackward>)
tensor(0.4455, grad_fn=<NllLossBackward>)
tensor(0.4383, grad_fn=<NllLossBackward>)
tensor(0.4326, grad_fn=<NllLossBackward>)
tensor(0.4265, grad_fn=<NllLossBackward>)
tensor(0.4200, grad_fn=<NllLossBackward>)
tensor(0.4122, grad_fn=<NllLossBackward>)
tensor(0.4043, grad_fn=<NllLossBackward>)
tensor(0.3946, grad_fn=<NllLossBackward>)
tensor(0.3829, grad_fn=<NllLossBackward>)
tensor(0.3687, grad_fn=<NllLossBackward>)
tensor(0.3482, grad_fn=<NllLossBackward>)
tensor(0.3254, grad_fn=<NllLossBackward>)
tensor(0.3060, grad_fn=<NllLossBackward>)
tensor(0.2842, grad_fn=<NllLossBackward>)
tensor(0.2399, grad_fn=<NllLossBackward>)
tensor(0.2054, grad_fn=<NllLossBac

In [212]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        X, y = data
        output = net(X.view(-1,7585))
        #print(output)
        for idx, i in enumerate(output):
            #print(torch.argmax(i), y[idx])
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1

print("Accuracy: ", round(correct/total, 3))

Accuracy:  0.408


<ipython-input-202-8e3d07ae977f>:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
